## demo

@todo: 

---

说明爬取哪里的数据（附上链接），得到数据包含什么。

举例：

爬取[肇庆市政府网页][肇庆市政府网页]中关于网友问政的内容

具体包含：问政标题、 网友名称、问政日期、浏览次数、问政内容、回复内容、牵头部门 共7项数据

[肇庆市政府网页]:http://wz.zhaoqing.gov.cn/wzpt/index.php?act=political&op=political_list1&curpage=0

---

### 1.import一些会使用到的库 + 使用到的函数

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import traceback
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'}

def getHTMLText(url, code='utf-8'):
    cnt=1
    '''测试版本的getHTMLText函数 只尝试 较少的次数 进行抓取，方便debug'''
    tryTime = 5
    while cnt<=tryTime:
        try:
            print("处理URL:"+url)
            r = requests.get(url,headers=headers,timeout=3000)
            r.raise_for_status()
            print('编码方式为：'+r.apparent_encoding)#注意判断编码方式
            r.encoding = code
            return r.text
        except:
            cnt=cnt+1
            traceback.print_exc()
            print('\n\ntry again....\n\n')
    print('%s times try, but failed........' % tryTime)
    return None

### 2.指定的要爬取的每一页的链接

In [7]:
basicUrl='xxx{page}' 

### 3.得到对应一个主页面（如首页）源代码下包含的所有子页面的链接，用list存储

In [11]:
#得到首页的源代码
htmlText=getHTMLText(basicUrl.format(page = 0)) ### 有时是0，有时是1，这个需要注意
#得到页面的子链接(一般用正则表达式来提取更快)
detailPattern=re.compile('political_id=(\d)+') ### 查看源代码，找到子链接的正则表达式
#使用列表表达式得到当前页面下的所有子页面的链接
detailURLs=['http://www.huizhou.gov.cn/viewletter.shtml?method=viewLetter&lid='+ id for id in detailPattern.findall(htmlText)]

### 4.得到最大页面数

In [12]:
def getMaxPage(basicUrl):
    print('预处理阶段：得到最大页面数')
    test = getHTMLText(basicUrl.format(page=0))
    '''
    使用正则表达式得到 【末页】 对应的最大页面
    方法：查看源代码->搜索 【末页】
    注意点：正则表达式要尽量精确，对于匹配最大页面数，使用\d+来匹配相对.*?就比较精确
    '''
    maxPagePattern = r'curpage=(\d+)\"><span>末页</span>'
    return re.findall(maxPagePattern,test)[0]
print(getMaxPage(basicUrl))

### 5.下面测试链接的抓取

In [13]:
# testLink1 = ''
# testText = getHTMLText(testLink1)
# '''有明显区分的规则的则使用re, 反之则使用BeautifulSoup'''
# soup1 = BeautifulSoup(testText, "html.parser")
# detailList = soup1.find_all('table',
#                           attrs={'width':'100%',
#                                  'border':'0',
#                                  'cellpadding':'3',
#                                  'cellspacing':'5'})
# #balabala